In [1]:
# display plots inline
%matplotlib notebook

# imports
import os
import numpy as np
import pandas as pd
import pymc3 as pm
from bambi import Model, Prior
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import pymc3_utils as pmu

# suppress system warnings for legibility
import warnings
warnings.filterwarnings('ignore')

# resize plots to fit labels inside bounding box
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# MPI color scheme
sns.set(style='white', palette='Set2')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Digit span
## Loading data

In [2]:
df_span = pd.read_csv('data/digit_span.tsv', sep='\t')
df_span = pd.melt(df_span, id_vars=['pp'], value_vars=['fds', 'bds'], var_name='task', value_name='span')
df_reading = pd.read_csv('data/tamil_reading.tsv', sep='\t')
df_span = df_span.merge(df_reading, left_on='pp', right_on='pp')

display(df_span.head())

,pp,task,span,subject,literate,word,pseudoword
0,low_1,fds,5,1,low,24,1
1,low_1,bds,4,1,low,24,1
2,y_2,fds,5,2,y,63,42
3,y_2,bds,5,2,y,63,42
4,y_3,fds,6,3,y,80,49


## Data mangling and plotting

In [3]:
# standardize variables
df_span['span_z'] = pmu.standardize(df_span['span'])
df_span['reading_z'] = pmu.standardize(df_span['word'])
df_span['task_binary'] = pd.get_dummies(df_span['task'])['fds']
df_span['task_z'] = pmu.standardize(df_span['task_binary'])

display(df_span.head())

,pp,task,span,subject,literate,word,pseudoword,span_z,reading_z,task_binary,task_z
0,low_1,fds,5,1,low,24,1,1.001956,-0.315292,1,0.997419
1,low_1,bds,4,1,low,24,1,0.358315,-0.315292,0,-0.997419
2,y_2,fds,5,2,y,63,42,1.001956,1.074860,1,0.997419
3,y_2,bds,5,2,y,63,42,1.001956,1.074860,0,-0.997419
4,y_3,fds,6,3,y,80,49,1.645596,1.680824,1,0.997419


## Modeling

First we will set some parameters for the regression procedure, as outlined in the paper.

In [4]:
# default model params
defaults = {
    'samples': 5000,
    'tune': 2500,
    'chains': 4,
    'init': 'advi+adapt_diag',
    'priors': {'fixed': 'narrow', 'random': 'narrow'},
}

We will use the only model that makes sense in the context of our multilevel model setup: A task model, an additional predictor for task type (fds versus bds) and by-participant intercepts.

In [5]:
model_span_task = Model(df_span)
model_span_task.fit('span_z ~ task_z',
                    random=['1|pp'],
                    **defaults)

Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...
Average Loss = 263.61:  29%|██▉       | 14599/50000 [00:06<00:16, 2169.56it/s]
Convergence achieved at 14600
Interrupted at 14,599 [29%]: Average Loss = 369.93
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [span_z_sd, 1|pp_offset, 1|pp_sd, task_z, Intercept]
Sampling 4 chains, 0 divergences: 100%|██████████| 30000/30000 [00:22<00:00, 1321.19draws/s]


In [6]:
display(pm.summary(model_span_task.backend.trace).round(2))

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
Intercept[0],0.00,0.08,-0.16,0.14,0.0,0.0,6931.0,6931.0,6927.0,11103.0,1.0
task_z[0],0.36,0.04,0.27,0.43,0.0,0.0,20778.0,20758.0,20784.0,14703.0,1.0
1|pp_offset[0],0.73,0.56,-0.30,1.83,0.0,0.0,19747.0,15509.0,19717.0,14032.0,1.0
1|pp_offset[1],0.04,0.57,-1.08,1.06,0.0,0.0,24944.0,8364.0,24942.0,14230.0,1.0
1|pp_offset[2],0.72,0.56,-0.34,1.76,0.0,0.0,26549.0,18194.0,26521.0,15129.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1|pp[93],0.47,0.37,-0.21,1.17,0.0,0.0,21228.0,15751.0,21241.0,13604.0,1.0
1|pp[94],0.25,0.36,-0.41,0.96,0.0,0.0,23112.0,11984.0,23097.0,14101.0,1.0
1|pp[95],0.47,0.36,-0.21,1.16,0.0,0.0,20694.0,15889.0,20684.0,14378.0,1.0
1|pp[96],0.02,0.36,-0.66,0.68,0.0,0.0,20047.0,9444.0,20042.0,14263.0,1.0


\\(\hat{r}\\) values look good, as does the number of effective samples for the by-participant intercepts. For the overall intercept we have more than 8K effective samples, for the span variance we have more than 5.5K.

## Plotting selected model traces

As an additional check, we will plot the posterior traces for the model. Ideally these look unimodal and roughly normal. The plots on the righthand side should look like fuzzy caterpillars.

In [7]:
g_traces = pm.traceplot(model_span_task.backend.trace)
plt.savefig('figures/digit_span_model_traces.png', dpi=600)

<IPython.core.display.Javascript object>

## Extracting participant intercept modes

We will now extract and store the posterior means of the participant intercepts so we can use them to model reading scores.

In [8]:
pps = df_span['pp'].unique()
pp_nums = [f'1|pp[{i}]' for i in range(len(pps))]
df_intercepts = pm.summary(model_span_task.backend.trace).loc[pp_nums]
df_intercepts['pp'] = np.sort(pps)

display(df_intercepts.head().round(2))

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat,pp
1|pp[0],0.47,0.36,-0.21,1.17,0.0,0.0,18294.0,14771.0,18296.0,14186.0,1.0,low_1
1|pp[1],0.02,0.36,-0.70,0.67,0.0,0.0,23909.0,8949.0,23941.0,14188.0,1.0,low_10
1|pp[2],0.47,0.36,-0.24,1.12,0.0,0.0,24364.0,18021.0,24357.0,14809.0,1.0,low_11
1|pp[3],0.25,0.36,-0.42,0.95,0.0,0.0,20385.0,12745.0,20375.0,14323.0,1.0,low_12
1|pp[4],0.69,0.36,0.01,1.38,0.0,0.0,19936.0,18007.0,19951.0,14717.0,1.0,low_13


In [9]:
df_uncorrected = df_span.groupby('pp', as_index=False).mean().rename(columns={'span': 'raw_span_mean'})
df_intercepts = df_intercepts[['pp', 'mean']].rename(columns={'mean': 'span_intercept'})
df_intercepts = df_intercepts.merge(df_uncorrected[['pp', 'reading_z', 'raw_span_mean']],
                                    left_on='pp', right_on='pp').reset_index()

display(df_intercepts.head().round(2))

,index,pp,span_intercept,reading_z,raw_span_mean
0,0,low_1,0.47,-0.32,4.5
1,1,low_10,0.02,0.65,3.5
2,2,low_11,0.47,0.36,4.5
3,3,low_12,0.25,0.01,4.0
4,4,low_13,0.69,0.01,5.0


In [10]:
# and write to file
df_intercepts.to_csv('data/span_intercepts.tsv', sep='\t')

Before closing this notebook, we will take a quick look at the correlations between working memory and reading score.

In [11]:
display(df_intercepts[['raw_span_mean', 'span_intercept', 'reading_z']].corr().round(2))

,raw_span_mean,span_intercept,reading_z
raw_span_mean,1.00,1.00,0.51
span_intercept,1.00,1.00,0.51
reading_z,0.51,0.51,1.00


Apparently it makes absolutely no difference whether we use the participant intercepts from our span model or the mean of forward and backward digit span for each individual participant, since those variables are perfectly correlated.